In [19]:
import os
import numpy as np
import pandas as pd

In [44]:
# 删除空数据
data_folder = r"./re_encode_data"
data_file_names = os.listdir(data_folder)
na_values = ["--"]
main_df = pd.DataFrame()
for data_file_name in data_file_names:
    data_file_path = os.path.join(data_folder, data_file_name)
    df = pd.read_csv(data_file_path, encoding="utf-8", index_col=0, na_values=na_values)
    df = df.dropna(axis=0,subset = ["评分"])
    city_name = data_file_name.split('.')[0]
    df["city_name"] = city_name
    main_df = pd.concat([main_df, df], axis=0)

In [101]:
# main_df.to_csv("main_df.csv", encoding="utf-8_sig", index=False)
main_df["评分"].astype(np.float32)
groups = main_df.groupby("评分")
best_score = 0
i = 0
for score, group in groups:
    if i == 0:
        best_score_citys = group
    if score > best_score:
        best_score = score
        best_score_citys = group
print(f"景点最高分（BS）是{best_score}")
print(f"获得BS（最高得分）的景点有{best_score_citys.shape[0]}个。")
best_score_city_groups = best_score_citys.groupby("city_name")
nsight_in_citys = []
citys = []
for city, group in best_score_city_groups:
    citys.append(city)
    nsight_in_citys.append(group.shape[0])
city_name_and_nsight = pd.DataFrame([citys, nsight_in_citys])
city_name_and_nsight = city_name_and_nsight.T
city_name_and_nsight.columns = ["city_name", "nsight"]
city_name_and_nsight = city_name_and_nsight.sort_values("nsight", ascending=False)
city_name_and_nsight.to_csv("./output/citys_and_nBS_sights.csv", encoding="utf-8_sig", index=False)
top_10_citys = list(city_name_and_nsight[0:10]["city_name"])
print(f"拥有BS（最高得分）景点数量最多的前10个城市为：{top_10_citys}")

景点最高分（BS）是5.0
获得BS（最高得分）的景点有2563个。
拥有BS（最高得分）景点数量最多的前10个城市为：['三沙', '五家渠', '玉溪', '益阳', '天门', '阿拉尔', '潍坊', '烟台', '大兴安岭', '邢台']
